In [27]:
import requests
import re
import cmipld

repo_url = cmipld.utils.git.url()

# GitHub repository and issue details
repo_owner = 'WCRP-CMIP'
repo_name = 'WCRP-universe'
issue_number = 2  # Replace with your issue number
# github_token = 'your-github-token'  # Replace with your GitHub token

# GitHub API URL for the issue
url = f'https://api.github.com/repos/{repo_owner}/{repo_name}/issues/{issue_number}'
# https://github.com/WCRP-CMIP/WCRP-universe/issues/2
# Headers for authentication
headers = {
    # 'Authorization': f'token {github_token}',
    'Accept': 'application/vnd.github.v3+json'
}

# Fetch the issue content
response = requests.get(url, headers=headers)
content = response.json()
issue = content.get('body', '')
labels_full = content.get('labels','')

# name: Read Issue Content and Labels

# on:
#   issues:
#     types: [opened, edited, labeled, unlabeled]

# jobs:
#   read-issue:
#     runs-on: ubuntu-latest
#     steps:
#       - name: Read Issue Content and Labels
#         uses: actions/github-script@v7
#         with:
#           github-token: ${{ secrets.GITHUB_TOKEN }}
#           script: |
#             const issue = context.payload.issue;
#             console.log(`Issue Title: ${issue.title}`);
#             console.log(`Issue Body: ${issue.body}`);
#             console.log(`Issue Labels: ${issue.labels.map(label => label.name).join(', ')}`);


In [2]:
import glob,os 
tloc = '../ISSUE_TEMPLATE/'

# templates = glob.glob('../ISSUE_TEMPLATE/*.yml')
# templates

In [ ]:
# labels = [i['name']for i in labels_full]

# if 'alpha' in labels:
#     labels.remove('alpha')
#     prefix='add_'
# elif 'delta' in labels:
#     labels.remove('delta')
#     prefix='modify_'
# else: 
#     prefix = ''
    
# # find the relevant form


In [5]:
issue

'### Issue Type\n\ninstitution\n\n### Acronym\n\ntest\n\n### Full name of the organisation.\n\ntest\n\n### ROR\n\n1w897891273\n\n### Other Notes\n\nhello'

In [28]:
import json

def parse_issue_body(issue_body):
    lines = issue_body.split('\n')
    issue_data = {}
    current_key = None

    for line in lines:
        if line.startswith('### '):
            current_key = line[4:].strip().replace(' ', '_').lower()
            issue_data[current_key] = ''
        elif current_key:
            issue_data[current_key] += line.strip() + ' '

    # Remove trailing spaces
    for key in issue_data:
        issue_data[key] = issue_data[key].strip()
    return issue_data
    # return json.dumps(issue_data, indent=4)

parsed_issue = parse_issue_body(issue)
print(json.dumps(parsed_issue,indent=4))

{
    "issue_type": "institution",
    "acronym": "UCLA",
    "full_name_of_the_organisation.": "University of California, Los Angeles",
    "ror": "046rm7j60",
    "other_notes": "reference issue on mip-cmor-tables by [znichollscr](https://github.com/znichollscr)"
}


In [31]:
import os

import importlib.util

# Extract the issue_type from the parsed issue
# issue_data = json.loads(parsed_issue)
issue_type = parsed_issue.get('issue_type', '')

path = '../ISSUE_SCRIPT/'


# Define the path to the script based on the issue_type
script_path = os.path.join(tloc, f"{path}{issue_type}.py")

# Check if the script exists
if os.path.exists(script_path):
    # Load the script dynamically
    spec = importlib.util.spec_from_file_location(issue_type, script_path)
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    print(f"Successfully loaded {script_path}")
else:
    print(f"Script {script_path} does not exist")
    
    
print(module)

module.run(parsed_issue,parsed_issue)

Successfully loaded ../ISSUE_TEMPLATE/../ISSUE_SCRIPT/institution.py
<module 'institution' from '/Users/daniel.ellis/WIPwork/WCRP-universe/.github/ISSUE_SCRIPT/../ISSUE_TEMPLATE/../ISSUE_SCRIPT/institution.py'>
issue {'issue_type': 'institution', 'acronym': 'UCLA', 'full_name_of_the_organisation.': 'University of California, Los Angeles', 'ror': '046rm7j60', 'other_notes': 'reference issue on mip-cmor-tables by [znichollscr](https://github.com/znichollscr)'}


In [25]:





# add sanitasion functons. 

ror_template = 'https://api.ror.org/organizations/{}'

url = ror_template.format(parsed_issue['ror'])

ror_data = cmipld.utils.read_url(url)

if ror_data:

    cmip_acronym = parsed_issue['acronym'].replace('_','-')
    
    ror_data =  {
        "id": f"{whoami}:organisations/{cmip_acronym.lower()}",
        "type": ['wcrp:organisation','wcrp:institution','universal'],
        "label": cmip_acronym,
        "ror": ror_data['id'].split('/')[-1],
        "long_label": ror_data['name'],
        "url": ror_data.get('links', []) ,
        "established": ror_data.get('established'),
        "kind": ror_data.get('types', [])[0] if ror_data.get('types') else None,
        "labels": [i['label'] for i in ror_data.get('lables', [])],
        "aliases": ror_data.get('aliases', []),
        "acronyms": ror_data.get('acronyms', []),
        "location": {
            "@id": f"universal:location/{ror_data['id'].split('/')[-1]}",
            "@type": "wcrp:location",
            "lat":  ror_data['addresses'][0].get('lat') if ror_data.get('addresses') else None,
            "lon":  ror_data['addresses'][0].get('lat') if ror_data.get('addresses') else None,
            "city": ror_data['addresses'][0].get('city') if ror_data.get('addresses') else None,
            "country": list(ror_data['country'].values())  if ror_data.get('country') else None
        
        }         
        #  can reverse match consortiums or members from here.    
        
    }

print(json.dumps(ror_data,indent=4))

make test for url2io
{
    "id": "universal:organisations/ucla",
    "type": [
        "wcrp:organisation",
        "wcrp:institution",
        "universal"
    ],
    "label": "UCLA",
    "ror": "046rm7j60",
    "long_label": "University of California, Los Angeles",
    "url": [
        "https://www.ucla.edu"
    ],
    "established": 1919,
    "kind": "Education",
    "labels": [],
    "aliases": [
        "State Normal School at Los Angeles",
        "University of California Southern Branch",
        "University of California at Los Angeles"
    ],
    "acronyms": [
        "UCLA"
    ],
    "location": {
        "@id": "universal:location/046rm7j60",
        "@type": "wcrp:location",
        "lat": 34.05223,
        "lon": 34.05223,
        "city": "Los Angeles",
        "country": [
            "United States",
            "US"
        ]
    }
}


In [ ]:
def similarity(name1, name2):
    from difflib import SequenceMatcher
   
    matcher = SequenceMatcher(None, name1, name2)
    similarity = matcher.ratio() * 100

    return similarity

